In [1]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

/home/ec2-user/miniconda/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


Loading BokehJS ...

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful.** 

<br />



- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b) Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.**

We first load in our data and come up with our priors for the parameters of the three distributions. We don't have much prior knowledge so we will keep them simple (normal distribution).

*Exponential(tao)*

tao ~ normal(700, 100)

*Gamma($\alpha$, $\beta$)*

alpha ~ normal(10, 3)

beta ~ normal(10, 3)

*Weibull($\alpha$, $\sigma$)*

$\alpha$ ~ normal(3, 0.05)

$\sigma$ ~ normal(10, 3)


In [5]:
df = pd.read_csv('/home/ec2-user/data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [82]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35.0,25.0,50.0,60.0
1,40.000,45.0,40.0,60.0,75.0
2,40.000,50.0,40.0,60.0,75.0
3,45.429,50.0,45.0,75.0,85.0
4,50.000,55.0,50.0,75.0,115.0


In [63]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')
prior_sm2 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m2.stan')
prior_sm3 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_084aff62b7267c6e5172712f7ec900d9 NOW.


Using cached StanModel.
Using cached StanModel.


/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmpaxme2ynu/stanfit4anon_model_084aff62b7267c6e5172712f7ec900d9_3780854271197521839.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [65]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen2 = prior_sm2.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen3 = prior_sm3.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen2 = bebi103.stan.to_dataframe(samples_gen2, diagnostics=False)
df_gen3 = bebi103.stan.to_dataframe(samples_gen3, diagnostics=False)

# Let's look at one of the dataframes to make sure they look away
df_gen3.head()

,chain,chain_idx,warmup,uM_12[1],uM_12[2],uM_12[3],uM_12[4],uM_12[5],uM_12[6],uM_12[7],...,uM_12[686],uM_12[687],uM_12[688],uM_12[689],uM_12[690],uM_12[691],uM_12[692],alpha,sigma,lp__
0,1,1,0,376.467016,2433.847279,704.482308,688.777455,2815.672401,538.901780,382.496489,...,1350.075999,939.277877,653.673705,422.402776,171.514060,244.055300,116.110145,0.974421,601.314697,0.0
1,1,2,0,386.273425,52.656172,46.989560,15.284868,121.779870,261.973964,119.625049,...,358.216759,460.849834,372.569606,660.170422,34.257408,139.381623,103.196077,0.947059,275.936338,0.0
2,1,3,0,77.134755,6.365506,433.036646,91.854794,241.603278,290.041921,128.494516,...,139.364940,381.313546,395.150512,1238.805969,641.533338,163.931697,5.617199,0.876617,310.724479,0.0
3,1,4,0,675.688129,2.473518,1.622834,112.055645,5.006407,347.054713,22.162829,...,51.277000,536.158793,350.815329,343.692630,18.637853,273.963639,692.298251,0.856552,259.107412,0.0
4,1,5,0,784.965555,347.516735,1947.447523,805.131347,192.047915,1354.688035,140.459471,...,668.878276,1961.011696,656.232064,340.005927,236.669448,131.154538,388.288050,0.932283,501.457494,0.0


In [66]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 6000)
bokeh.io.show(p)

In [67]:
p = bebi103.viz.predictive_ecdf(samples_gen2, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 3000)
bokeh.io.show(p)

In [69]:
p = bebi103.viz.predictive_ecdf(samples_gen3, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 1000)
bokeh.io.show(p)

Our priors look reasonable, so now we can move on to creating our mcmc models and sampling. **Actually, they really don't look fine, but I guess we'll worry about that later**

In [70]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b24f98775e1f7817b1462e229c94c937 NOW.


Using cached StanModel.
Using cached StanModel.


/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmpet177prs/stanfit4anon_model_b24f98775e1f7817b1462e229c94c937_8425740589495078112.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [71]:
data = dict(N=len(df),
           uM_12=df['12 uM'].values.astype(float))

In [80]:
samples1 = sm1.sampling(data=data)
samples2 = sm2.sampling(data=data)
samples3 = sm3.sampling(data=data)

In [81]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)
df_mcmc2 = bebi103.stan.to_dataframe(samples2, diagnostics=False, inc_warmup=False)
df_mcmc3 = bebi103.stan.to_dataframe(samples3, diagnostics=False, inc_warmup=False)

In [74]:
df_mcmc3.head()

,chain,chain_idx,warmup,alpha,sigma,log_like[1],log_like[2],log_like[3],log_like[4],log_like[5],...,uM_12_ppc[684],uM_12_ppc[685],uM_12_ppc[686],uM_12_ppc[687],uM_12_ppc[688],uM_12_ppc[689],uM_12_ppc[690],uM_12_ppc[691],uM_12_ppc[692],lp__
0,1,1,0,1.593461,411.542512,-7.227834,-6.961751,-6.961751,-6.891700,-6.839730,...,373.330425,671.833863,222.466581,327.523892,85.131024,829.734022,221.320810,191.208338,527.332540,-4678.106546
1,1,2,0,1.579319,409.391789,-7.189427,-6.930451,-6.930451,-6.862374,-6.811910,...,114.869690,377.505517,244.494799,174.375331,637.017466,438.077695,374.602473,307.891502,134.428071,-4678.435168
2,1,3,0,1.544079,416.429104,-7.140691,-6.898827,-6.898827,-6.835411,-6.788463,...,247.993770,582.703520,218.317465,302.205684,336.366937,294.081792,110.742105,415.076174,851.758383,-4679.214064
3,1,4,0,1.548457,428.458754,-7.193558,-6.948928,-6.948928,-6.884665,-6.837039,...,217.383140,319.727764,915.243489,219.328313,182.655733,144.897311,274.589041,226.720366,687.583628,-4679.826380
4,1,5,0,1.593265,416.801822,-7.247416,-6.981172,-6.981172,-6.911038,-6.858990,...,271.506181,205.634767,197.223192,36.279629,444.520479,389.134129,319.560279,58.017235,400.718818,-4677.885381


We now plot the post predictive checks to do model comparison.

In [75]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples1, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

We can see there is a systematic issue with using the exponential distribution becuase the two inflection points will always be in the wrong place.

In [76]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples2, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

The gamma distribution looks a lot more promising!

In [77]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples3, 
                                          percentiles=[99, 70, 50, 30],
                                          name='uM_12_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

Ooof this is not great (better than exponential though). Only a few of our datapoints fall in that 99% confidence interval.

This doesn't give us a whole lot of information, so we compute the loo given the log likelihood and use the bebi103 compare function to calculate the loo and the weights.

In [78]:
bebi103.stan.compare({'exponential': samples1, 'gamma': samples2, 'weibull': samples3},
                     log_likelihood='log_like', ic='loo')

,loo,ploo,dloo,weight,se,dse,warning
gamma,9281.06,2.4435,0,0.958793,48.172,0,0
weibull,9321.51,2.01426,40.4515,1.04906e-13,43.0174,11.8882,0
exponential,9608.59,0.368836,327.524,0.0412069,31.9052,31.6414,0


We can see that gamma (Model 2) is the best distribution to fit our data because the weight is much higher (0.95 vs 10^-2 and 10^-13). Based on the visual analysis of the post predictive checks above,  this makes a lot of sense! 

**c) Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?**
